In [6]:
import torch
import torch.nn
from torchvision.models import convnext_large
from torch.optim import SGD, Adam
from torchvision import transforms
from torch.utils.data import Dataset, DataLoader, Subset
from PIL import Image
import os

In [7]:
transform = transforms.Compose([
                                transforms.Resize([232], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([224,224]),
                                transforms.ToTensor(), # Rescaled to 0-1 aswell
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
])

target_transform = transforms.Compose([
                                transforms.Resize([232], interpolation=transforms.InterpolationMode.BILINEAR),
                                transforms.CenterCrop([224,224]),
                                transforms.ToTensor(),
])

In [19]:
class CustomDataset(Dataset):
    def __init__(self, path, transform=None, target_transform=None):
        self.transform= transform
        self.target_transform = target_transform
        self.path = path
        self.labels = []
        self.images = []
        self.count = 0
        for i, direc in enumerate(os.listdir(self.path)[1:]): # [1:] because MacOS has .DS_Store at 0th index
            for j, file_name in enumerate(os.listdir(os.path.join(self.path,direc))):
                self.labels.append(i)
                image_path = os.path.join(self.path, direc ,file_name)
                image = Image.open(image_path)
                image = transform(image)
                self.images.append(image)
        
        # Early loading
        
        
    def __len__(self):
        return len(self.labels)
    
    def __getitem__(self, idx):
        # Lazy loading
        
#         directory_idx = idx//100
#         file_idx = idx%100
#         label = directory_idx

        return self.images[idx], self.labels[idx]
        

In [20]:
dataset = CustomDataset('tiny_FR', transform)

In [21]:
torch.manual_seed(10)
index = torch.randperm(500)

In [22]:
data_train = Subset(dataset,index[:400])
data_valid = Subset(dataset,index[400:])

In [23]:
torch.manual_seed(1)
data = DataLoader(data_train, 32, shuffle=True)
data_vl = DataLoader(data_valid, 4, shuffle=True)

In [24]:
next(iter(data))[0].shape

torch.Size([32, 3, 224, 224])

In [38]:
model = convnext_large(weights='IMAGENET1K_V1')

In [39]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [40]:
for param in model.parameters():
    param.requires_grad = False

In [41]:
num_ftrs = model.classifier[2].in_features
model.classifier[2] = torch.nn.Linear(num_ftrs, 5)

In [42]:
model

ConvNeXt(
  (features): Sequential(
    (0): Conv2dNormActivation(
      (0): Conv2d(3, 192, kernel_size=(4, 4), stride=(4, 4))
      (1): LayerNorm2d((192,), eps=1e-06, elementwise_affine=True)
    )
    (1): Sequential(
      (0): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(in_features=192, out_features=768, bias=True)
          (4): GELU(approximate='none')
          (5): Linear(in_features=768, out_features=192, bias=True)
          (6): Permute()
        )
        (stochastic_depth): StochasticDepth(p=0.0, mode=row)
      )
      (1): CNBlock(
        (block): Sequential(
          (0): Conv2d(192, 192, kernel_size=(7, 7), stride=(1, 1), padding=(3, 3), groups=192)
          (1): Permute()
          (2): LayerNorm((192,), eps=1e-06, elementwise_affine=True)
          (3): Linear(

In [43]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.classifier[2].parameters(), lr=0.001, momentum=0.9)

In [44]:
def train(epoch, data, data_vl):
    train_loss = [0] * epoch
    train_accuracy = [0] * epoch
    
    valid_loss = [0] * epoch
    valid_accuracy = [0] * epoch
    model.train()
    for i in range(epoch):
        for x_batch, y_batch in data:
            #print(x_batch.shape)
            
            pred = model(x_batch)
            loss = loss_fn(pred, y_batch)
            loss.backward()
            optimizer.step()
            optimizer.zero_grad()
            
            train_loss[i] += loss.item() * x_batch.size(0)
            correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
            train_accuracy[i] += correct.to(torch.float32).item()
        train_loss[i] /= len(data.dataset)
        train_accuracy[i] /= len(data.dataset)
        
        with torch.no_grad():
            for x_batch, y_batch in data_vl:
                pred = model(x_batch)
                loss = loss_fn(pred, y_batch)
                valid_loss[i] += loss.item() * x_batch.size(0)
                correct = torch.sum(torch.argmax(torch.softmax(pred,axis=1), axis=1) == y_batch)
                valid_accuracy[i] += correct.item()
            valid_loss[i] /= len(data_vl.dataset)
            valid_accuracy[i] /= len(data_vl.dataset)
        print(f'Epoch {i+1} accuracy: {train_accuracy[i]:.4f} val_accuracy:{valid_accuracy[i]:.4f}')
        print(f'Epoch {i+1} loss: {train_loss[i]:.4f} val_loss:{valid_loss[i]:.4f}')
        print()
            

In [45]:
train(50, data, data_vl)

Epoch 1 accuracy: 0.2900 val_accuracy:0.5200
Epoch 1 loss: 1.5572 val_loss:1.4573




KeyboardInterrupt



In [ ]:
from sklearn.metrics import f1_score, confusion_matrix